# Imports

In [2]:
from dash import Dash, dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template
import plotly.express as px
import pandas
import numpy as np

# df_edu

In [3]:
df_edu = pandas.read_csv('data/states_all.csv').iloc[:, 1:]
df_edu.head()

,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,735036.0,...,57948.0,58025.0,41167.0,NaN,NaN,731634.0,208.0,252.0,207.0,NaN
1,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,350902.0,...,9748.0,8789.0,6714.0,NaN,NaN,122487.0,NaN,NaN,NaN,NaN
2,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,1007732.0,...,55433.0,49081.0,37410.0,NaN,NaN,673477.0,215.0,265.0,209.0,NaN
3,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,483488.0,...,34632.0,36011.0,27651.0,NaN,NaN,441490.0,210.0,256.0,211.0,NaN
4,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,8520926.0,...,418418.0,363296.0,270675.0,NaN,NaN,5254844.0,208.0,261.0,202.0,NaN


# df_ski

In [12]:
df_ski = pandas.read_csv('data/European_Ski_Resorts.csv')  #.drop("Unnamed: 0", axis=1)

# Data Table 1

In [13]:
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
app = Dash(__name__, )  # external_stylesheets=[dbc.themes.BOOTSTRAP]

app.layout = html.Div(
    dash_table.DataTable(
        columns=[{"name": i, "id": i} for i in df_edu.columns],
        data=df_edu.to_dict('records'),
        filter_action="native",
        sort_action="native",
        export_format="csv",
        style_header={
            'backgroundColor': 'rgb(30, 30, 30)',
            'fontWeight': 'bold',
            'font-family': 'Arial',
            'color': 'lightgrey'
        },
        style_data={
            'font-family': 'Arial',
            'color': 'grey',
            'backgroundColor': 'rgb(50, 50, 50)',
        }
    )
)

app.run_server(debug=True)


# ASSIGNMENT: Data Table

In [ ]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H2(id="title", style={"text-align": "center"}),
    html.P("Select Country Below:"),
    dcc.Dropdown(
        id="country_dropdown",
        options=df_ski['Country'].unique(),
        value="Andorra",
        placeholder="Select a country",
        className="dbc"
    ),
    html.Br(), html.Br(),
    html.P("Select Elevation:"),
    dcc.Slider(
        id="elevation_slider",
        min=0,
        max=4000,
        step=500,
        value=500,
        marks={i: {"label": f"{i} m", "style": {"fontSize": 16}}
               for i in range(0, 4001, 500)},
        className="dbc"
    ),
    html.Br(), html.Br(),
    html.Div(id="output_div")

])


@app.callback(
    Output("title", "children"),
    Output("output_div", "children"),
    Input("country_dropdown", "value"),
    Input("elevation_slider", "value"),
)
def elevation_table(country: str, elevation: int):
    if not country and not elevation:
        raise PreventUpdate

    df = df_ski.query("HighestPoint > @elevation and Country == @country")

    title = f"Ski resorts in {country} with elevation > {elevation}"

    table = dash_table.DataTable(
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        filter_action="native",
        sort_action="native",
        export_format="csv",
        style_header={
            'backgroundColor': 'rgb(30, 30, 30)',
            'fontWeight': 'bold',
            'font-family': 'Arial',
            'color': 'lightgrey'
        },
        style_data={
            'font-family': 'Arial',
            'color': 'grey',
            'backgroundColor': 'rgb(50, 50, 50)',
        }
    )

    return title, table


app.run_server(debug=True)

# Conditional Callbacks

In [24]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

df_conditional = (
    pandas.read_csv("data/states_all.csv")
    .iloc[:, 1:]
    .groupby("STATE", as_index=False)
    .agg({"TOTAL_EXPENDITURE": "mean"})
    .sort_values("TOTAL_EXPENDITURE", ascending=False)
)

app.layout = html.Div([
    dcc.RadioItems(
        id="output_picker",
        options=["bar", "table"],
        value="bar",
    ),
    html.Div(id="output_div")
])


@app.callback(
    Output(component_id="output_div", component_property="children"),
    Input(component_id="output_picker", component_property="value")
)
def output_generator(output_style: str):
    if output_style == "bar":
        output = dcc.Graph(figure=px.bar(df_conditional, x="STATE", y="TOTAL_EXPENDITURE"))
    else:
        output = dash_table.DataTable(
            columns=[{"name": i, "id": i} for i in df_conditional.columns],
            data=df_conditional.to_dict('records'),
            filter_action="native",
            sort_action="native",
            export_format="csv",
            style_header={
                'backgroundColor': 'rgb(30, 30, 30)',
                'fontWeight': 'bold',
                'font-family': 'Arial',
                'color': 'lightgrey'
            },
            style_data={
                'font-family': 'Arial',
                'color': 'grey',
                'backgroundColor': 'rgb(50, 50, 50)',
            }
        )

    return output


app.run_server(debug=True)


# Chained Callbacks

In [25]:
app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE])

df_edu_chain = (
    df_edu.assign(
        Avg_Math=lambda df: df[['AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE']].mean(axis=1),
        Avg_Reading=lambda df: df[['AVG_READING_4_SCORE', 'AVG_READING_8_SCORE']].mean(axis=1),
    )
)

# TODO: Add chained callbacks here
all_options = {}

,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,...,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,Avg_Math,Avg_Reading
0,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,735036.0,...,41167.0,NaN,NaN,731634.0,208.0,252.0,207.0,NaN,230.0,207.0
1,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,350902.0,...,6714.0,NaN,NaN,122487.0,NaN,NaN,NaN,NaN,NaN,NaN
2,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,1007732.0,...,37410.0,NaN,NaN,673477.0,215.0,265.0,209.0,NaN,240.0,209.0
3,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,483488.0,...,27651.0,NaN,NaN,441490.0,210.0,256.0,211.0,NaN,233.0,211.0
4,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,8520926.0,...,270675.0,NaN,NaN,5254844.0,208.0,261.0,202.0,NaN,234.5,202.0


# Interactive Cross-filtering

In [28]:
app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE])

df_cf = (
    df_edu
    .rename({
        "AVG_MATH_4_SCORE": "Math 4 grade",
        "AVG_MATH_8_SCORE": "Math 8 grade",
        "AVG_READING_4_SCORE": "Reading 4 grade",
        "AVG_READING_8_SCORE": "Reading 8 grade",
    }, axis=1)
    .assign(expenditure_per_student=lambda df: df['TOTAL_EXPENDITURE'] / df['GRADES_ALL_G'])
)

app.layout = html.Div([
    dbc.Row(html.H1("Education Performance & Expenditure", style={"text-align": "center"})),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dcc.Markdown("Select an X Column"),
                dcc.RadioItems(
                    id="score-radio",
                    options=[
                        {"label": "Math 4 grade", "value": "Math 4 grade"},
                        {"label": "Math 8 grade", "value": "Math 8 grade"},
                        {"label": "Reading 4 grade", "value": "Reading 4 grade"},
                        {"label": "Reading 8 grade", "value": "Reading 8 grade"},
                    ],
                    value="Math 8 grade",
                ),
                html.Hr(),
                dcc.Markdown("Select a Y Column"),
                dcc.RadioItems(
                    id="score-radio2",
                    options=[
                        {"label": "Math 4 grade", "value": "Math 4 grade"},
                        {"label": "Math 8 grade", "value": "Math 8 grade"},
                        {"label": "Reading 4 grade", "value": "Reading 4 grade"},
                        {"label": "Reading 8 grade", "value": "Reading 8 grade"},
                    ],
                    value="Math 8 grade",
                )
            ])
        ], width=2),
        dbc.Col(
            dcc.Graph(id="cross-filter-scatter", hoverData={"points": [{"customdata": "CALIFORNIA"}]}),
        ),
        dbc.Col(dcc.Graph(id="x-line"))
    ]),
    dbc.Row(html.P("Third row of app Layout")),
])

@app.callback(
    Output("cross-filter-scatter", "figure"),
    Input("score-radio", "value"),
    Input("score-radio2", "value"),
)
def score_scatter(x, y):
    fig = px.scatter(
        df_cf.query("YEAR == 2013"),
        x=x,
        y=y,
        hover_name="STATE",
        custom_data=["STATE"],
    )
    
    return fig 

@app.callback(
    Output("x-line", "figure"),
    Input("cross-filter-scatter", "hoverData")
)
def update_line(hover_data: dict):
    state_name = hover_data['points'][0]['customdata'][0]
    df = df_cf.query("STATE == @state_name")
    
    fig = px.line(
        df,
        x="YEAR",
        y="expenditure_per_student",
        title=f"Expenditure per student in {state_name.title()}",
    ).update_xaxes(showgrid=False)
    
    return fig
    
if __name__ == "__main__":
    app.run_server(debug=True)